# Libraries

In [65]:
import os
import json
import copy
import random

import pandas as pd
import numpy as np

import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset, ConcatDataset

from sklearn.metrics import f1_score

import transformers
from transformers import AutoModel, AutoTokenizer, BatchEncoding
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW

from typing import Optional, Dict, Tuple, List, Callable
from dataclasses import dataclass, field

import re
import emoji

# Configurations, Global Variables

In [66]:
import torch.backends.cudnn as cudnn

torch.manual_seed(421)
torch.cuda.manual_seed(421)
torch.cuda.manual_seed_all(421)
np.random.seed(421)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(421)

In [67]:
@dataclass
class DataArgs:
  train_dataset_path: str = '../data/nikluge-sa-2022-train.jsonl'
  # train_dataset_path: str = '../data/nikluge-sa-2022-train_augmented.jsonl'
  augmented_dataset_path: str = '../data/for_cd_train.jsonl'
  val_dataset_path: str = '../data/nikluge-sa-2022-dev.jsonl'
  test_dataset_path: str = '../data/nikluge-sa-2022-test.jsonl'
  batch_size: int = 256

In [86]:
@dataclass
class ModelArgs:
  max_tokens: int = 32
  huggingface_baseline: str = 'klue/roberta-small' #"beomi/KcELECTRA-base-v2022"
  dropout: float = 0.5

In [115]:
@dataclass
class TrainingArgs:
  n_epochs: int = 25
  lr: float = 2e-5
  save_path: str = '../saved_model/'
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [116]:
category_main_id2name = ['패키지/구성품', '제품 전체', '본품', '브랜드']
category_main_name2id = {category_main_id2name[i]: i for i in range(len(category_main_id2name))}

category_sub_id2name = ['품질', '다양성', '인지도', '편의성', '일반', '가격', '디자인']
category_sub_name2id = {category_sub_id2name[i]: i for i in range(len(category_sub_id2name))}

category_id2name = ['제품 전체#일반', '제품 전체#가격', '제품 전체#디자인', '제품 전체#품질', '제품 전체#편의성', '제품 전체#인지도', # 6
                    '본품#일반', '본품#디자인', '본품#품질', '본품#편의성', '본품#다양성', '본품#가격', '본품#인지도', # 7
                    '패키지/구성품#일반', '패키지/구성품#디자인', '패키지/구성품#품질', '패키지/구성품#편의성', '패키지/구성품#다양성', '패키지/구성품#가격', '패키지/구성품#인지도', # 7
                    '브랜드#일반', '브랜드#가격', '브랜드#디자인', '브랜드#품질', '브랜드#인지도'] # 5

category_name2id = {category_id2name[i]: i for i in range(len(category_id2name))}

polarity_id2name = ['positive', 'negative', 'neutral']
polarity_name2id = {polarity_id2name[i]: i for i in range(len(polarity_id2name))}

special_tokens_dict = {
  'additional_special_tokens': ['<name>', '<affiliation>', '<social-security-num>', '<tel-num>', '<card-num>', '<bank-account>', '<num>', '<online-account>'] + ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
    # 'additional_special_tokens': ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
}

# Dataset, DataLoader

In [117]:
class CustomDataset(Dataset):
  def __init__(
    self,
    path: str,
    tokenizer: transformers.PreTrainedTokenizer,
    max_tokens: Optional[int] = 128,
    is_test: bool = False
      ):
    self.is_test = is_test
    self.data = self._data_preprocess(self._load_data(path, self.is_test))
    self.tokenizer = tokenizer
    self.max_tokens = max_tokens


  def __len__(self) -> int:
    return len(self.data)


  def __getitem__(self, idx: int) -> dict:
    row = self.data.iloc[idx, :]
    inputs = self.tokenizer(row['sentence_form'], max_length=self.max_tokens, truncation=True, padding='max_length')
    input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
    
    if self.is_test:
      return {
          'input_ids': torch.LongTensor(input_ids),
          'attention_mask': torch.LongTensor(attention_mask)
          }
    else:
      labels = self._convert_to_binary(list(map(lambda x: category_name2id[x], row['category'])), n_classes=len(category_id2name))
      return {
          'input_ids': torch.LongTensor(input_ids),
          'attention_mask': torch.LongTensor(attention_mask),
          'labels': labels
          }


  def _convert_to_binary(self, labels: List[int], n_classes: int) -> torch.Tensor:
    binary_labels = torch.zeros(n_classes)
    binary_labels[labels] = 1
    return binary_labels


  def _load_json(self, path: str, encoding: Optional[str]="utf-8") -> list:
    with open(path, encoding=encoding) as f:
        json_list = [json.loads(line) for line in f.readlines()]
    return json_list


  def _load_data(self, path: str, is_test):
    if self.is_test:
      df = pd.DataFrame(self._load_json(path))
    else:
      df = pd.DataFrame(self._load_json(path)).explode('annotation')
      df['category'] = df['annotation'].apply(lambda x: x[0])
      df['polarity'] = df['annotation'].apply(lambda x: x[2])
      df = df.groupby('sentence_form').agg({'category': list, 'polarity': list}).reset_index()
    
    return df
  
  
  def _text_normalize(self, x):
    count = {}
    normalized_text = ""
    for char in x:
      if char not in count:
        count = {char: 1}
      else:
        count[char] += 1
      if count[char] <= 2:
        normalized_text += char
    return normalized_text
  
  
  def _text_preprocess(self, x):
    emojis = ''.join(emoji.EMOJI_DATA.keys())
    pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+') # {emojis}
    x = pattern.sub('', x) # 불용어 처리 kcElectra
    x = re.sub(' +', ' ', x) # 더블 스페이스 제거
    x = self._text_normalize(x)
    return x
  
  
  def _data_preprocess(self, df):
    df['sentence_form'] = df['sentence_form'].apply(self._text_preprocess)
    return df

In [118]:
class DataIterator():
  def __init__(
      self,
      data_args: DataArgs,
      model_args: ModelArgs,
      tokenizer: transformers.PreTrainedTokenizer
      ):
#     full_dataset = CustomDataset(data_args.augmented_dataset_path, tokenizer, max_tokens=model_args.max_tokens)
    
#     train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [0.8, 0.2])
    train_dataset = CustomDataset(data_args.train_dataset_path, tokenizer, max_tokens=model_args.max_tokens)
    test_dataset = CustomDataset(data_args.val_dataset_path, tokenizer, max_tokens=model_args.max_tokens)
    
    self.train = DataLoader(
        train_dataset,
        batch_size=data_args.batch_size,
        shuffle=True,
        drop_last=True
        )

    self.val = DataLoader(
        test_dataset,
        batch_size=data_args.batch_size,
        shuffle=False,
        drop_last=True
        )

# Model


## Layers

In [119]:
class MultiLabelClassifierLayer(nn.Module):
    def __init__(self, hidden_dim: int, n_label: int, model_args: ModelArgs):
      super(MultiLabelClassifierLayer, self).__init__()
      self.linear = nn.Linear(hidden_dim, hidden_dim)
      self.batch_norm = torch.nn.BatchNorm1d(hidden_dim)
      self.fc_out = nn.Linear(hidden_dim, n_label)
      self.relu = torch.nn.ReLU()
      self.dropout = nn.Dropout(model_args.dropout)

    def forward(self, x):
      x = self.linear(x)
      if x.dim() == 1:
        x = x.unsqueeze(0)
      x = self.batch_norm(x)
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc_out(x)
      return x

In [120]:
class AttentionPoolingLayer(nn.Module):
  def __init__(self, hidden_dim: int):
    super(AttentionPoolingLayer, self).__init__()
    self.linear = torch.nn.Linear(hidden_dim, 1)

  def forward(self, x):
    x = torch.stack([x[i][:, 0, :].squeeze() for i in range(1, len(x))])
    attention = F.softmax(self.linear(x), dim=0)
    x = torch.sum(attention * x, dim=0)
    return x

In [121]:
class ClassifierBlock(nn.Module):
    def __init__(self, hidden_dim: int, n_label: int, model_args: ModelArgs):
      super(ClassifierBlock, self).__init__()
      self.attn_pooling = AttentionPoolingLayer(hidden_dim)
      self.classifiers = nn.ModuleList([MultiLabelClassifierLayer(hidden_dim, 1, model_args) for _ in range(n_label)])

    def forward(self, x):
      x = self.attn_pooling(x)
      x = torch.cat([classifier(x) for classifier in self.classifiers], -1)
      return x

## Main

In [122]:
class CustomModel(nn.Module):
    def __init__(
        self,
        model_args: ModelArgs,
        len_tokenizer: int
        ):
        super(CustomModel, self).__init__()

        self.roberta = AutoModel.from_pretrained(model_args.huggingface_baseline)
        self.roberta.resize_token_embeddings(len_tokenizer)
        
        self.classifier_1 = ClassifierBlock(self.roberta.config.hidden_size, 6, model_args)
        self.classifier_2 = ClassifierBlock(self.roberta.config.hidden_size, 7, model_args)
        self.classifier_3 = ClassifierBlock(self.roberta.config.hidden_size, 7, model_args)
        self.classifier_4 = ClassifierBlock(self.roberta.config.hidden_size, 5, model_args)

    def forward(
        self,
        input_ids,
        attention_mask,
        **kwargs
        ):
      outputs = self.roberta(
          input_ids=input_ids,
          attention_mask=attention_mask,
          output_hidden_states=True,
          token_type_ids=None
          )

      hidden_states = outputs.hidden_states
      outputs = torch.cat((self.classifier_1(hidden_states), 
                           self.classifier_2(hidden_states), 
                           self.classifier_3(hidden_states),
                           self.classifier_4(hidden_states)), -1)

      return outputs

# Trainer

In [123]:
from sklearn.metrics import precision_score, recall_score

class MetricCalculator():
  def __init__(self, threshold):
    self.threshold = threshold
    self.TP = 0.001
    self.FP = 0.001
    self.FN = 0.001


  def print(self):
    precision = self.TP / (self.TP + self.FP)
    recall = self.TP / (self.TP + self.FN)
    print(f'국립국어원 기준 | Threshold: {self.threshold} | # Precision : {precision:.4f} | Recall : {recall:.4f} | F1 : {2 * precision * recall / (precision + recall):.4f}')


  def clean(self):
    self.TP = 0.001
    self.FP = 0.001
    self.FN = 0.001


  def calc(self, y_prob, y_true):
    y_pred = (torch.sigmoid(y_prob).cpu() > self.threshold)
    y_true = y_true.cpu()

    for i in range(len(y_pred)):
      a = set(np.where(y_pred[i])[0].tolist())
      b = set(np.where(y_true[i])[0].tolist())
      self.TP += len(set.intersection(a, b))
      self.FP += len(a - b)
      self.FN += len(b - a)

In [124]:
class Trainer():
    def __init__(self, model: nn.Module, optimizer: torch.optim, data_iterator: DataIterator, loss_fn, training_args: TrainingArgs,
                 scheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None):
        self.device = training_args.device
        self.model = model.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler if scheduler else None
        self.data_iterator = data_iterator
        self.loss_fn = loss_fn
        self.train_loss = []
        self.test_loss = []
        
        self.metric_calculator_065 = MetricCalculator(0.65)
        self.metric_calculator_07 = MetricCalculator(0.7)
        self.metric_calculator_075 = MetricCalculator(0.75)
        self.metric_calculator_08 = MetricCalculator(0.8)

        self.save_path = training_args.save_path


    def fit(self, epochs):
        for epoch in range(1, epochs + 1):
            self.train(epoch)
            self.val(epoch)


    def train(self, epoch):
        self._train_step(epoch, self.data_iterator.train)


    def val(self, epoch):
        self._val_step(epoch, self.data_iterator.val)


    def _train_step(self, epoch, data_loader):
        data_iter = tqdm.tqdm(enumerate(data_loader), desc=f"train #{epoch}",
                              total=len(data_loader), bar_format="{l_bar}{bar:30}{r_bar}{bar:-30b}")
        self.model.train()
        running_loss = []
        for i, data in data_iter:
            data = {k: v.to(self.device) for k, v in data.items()}
            labels = data['labels']
            logits = self.model.forward(**data)
            loss = self.loss_fn(logits, labels)
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            if self.scheduler: self.scheduler.step()
            running_loss.append(loss.item())
            data_iter.set_postfix({
                "epoch": epoch,
                "iteration": i,
                "loss": sum(running_loss) / len(running_loss)})
            
            self.metric_calculator_065.calc(logits, labels)
            self.metric_calculator_07.calc(logits, labels)
            self.metric_calculator_075.calc(logits, labels)
            self.metric_calculator_08.calc(logits, labels)
            
        self.train_loss += running_loss

        self.metric_calculator_065.print()
        self.metric_calculator_065.clean()

        self.metric_calculator_07.print()
        self.metric_calculator_07.clean()
        
        self.metric_calculator_075.print()
        self.metric_calculator_075.clean()

        self.metric_calculator_08.print()
        self.metric_calculator_08.clean()



    def _val_step(self, epoch, data_loader):
        data_iter = tqdm.tqdm(enumerate(data_loader), desc=f"val #{epoch}",
                              total=len(data_loader), bar_format="{l_bar}{bar:30}{r_bar}{bar:-30b}")
        self.model.eval()
        running_loss = []
        for i, data in data_iter:
            data = {k: v.to(self.device) for k, v in data.items()}
            labels = data['labels']
            with torch.no_grad():
                logits = self.model.forward(**data)
                loss = self.loss_fn(logits, labels)
            running_loss.append(loss.item())
            data_iter.set_postfix({
                "epoch": epoch,
                "iteration": i,
                "loss": sum(running_loss) / len(running_loss)})
            self.metric_calculator_065.calc(logits, labels)
            self.metric_calculator_07.calc(logits, labels)
            self.metric_calculator_075.calc(logits, labels)
            self.metric_calculator_08.calc(logits, labels)

        running_loss = sum(running_loss) / len(running_loss)
        if True:# not running_loss or (running_loss < (min(self.test_loss) if self.test_loss else 0)):
            self.save(epoch, self.save_path, f"{running_loss:.3}")
            pass

        self.test_loss.append(running_loss)

        self.metric_calculator_065.print()
        self.metric_calculator_065.clean()

        self.metric_calculator_07.print()
        self.metric_calculator_07.clean()
        
        self.metric_calculator_075.print()
        self.metric_calculator_075.clean()

        self.metric_calculator_08.print()
        self.metric_calculator_08.clean()


    def save(self, epoch, file_path, loss):
        file_path = os.path.join(file_path, f"cd_new{loss}.pt")
        torch.save(self.model.state_dict(), file_path)
        self.model.to(self.device)
        print(f"EP:{epoch} | Model Saved on: {file_path}")

In [162]:
model_args = ModelArgs()
data_args = DataArgs()
training_args = TrainingArgs()

tokenizer = AutoTokenizer.from_pretrained(model_args.huggingface_baseline)
tokenizer.add_special_tokens(special_tokens_dict)

model = CustomModel(model_args, len_tokenizer = len(tokenizer))
optimizer = AdamW(params=filter(lambda x: x.requires_grad, model.parameters()), lr=training_args.lr)
data_iterator = DataIterator(data_args, model_args, tokenizer)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [163]:
print(len(data_iterator.train.dataset))
print(len(data_iterator.val.dataset))

count = []
for x in data_iterator.train.dataset:
  count.append(x['labels'].tolist())
count = torch.Tensor(list(map(lambda x: sum(x), zip(*count))))

sample_per_cls = torch.Tensor(count + np.ones(len(category_id2name)))
print([category_id2name[x] for x in np.where(np.array(sample_per_cls < 200))[0].tolist()])

3000
2793
['제품 전체#가격', '제품 전체#디자인', '제품 전체#편의성', '제품 전체#인지도', '본품#디자인', '본품#편의성', '본품#다양성', '본품#가격', '본품#인지도', '패키지/구성품#일반', '패키지/구성품#디자인', '패키지/구성품#품질', '패키지/구성품#편의성', '패키지/구성품#다양성', '패키지/구성품#가격', '패키지/구성품#인지도', '브랜드#일반', '브랜드#가격', '브랜드#디자인', '브랜드#품질', '브랜드#인지도']


In [164]:
class FocalLossWithPosWeight(nn.Module):
    def __init__(self, alpha=1, gamma=2, pos_weight=None, reduction='mean'):
        super(FocalLossWithPosWeight, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.pos_weight = pos_weight
        self.reduction = reduction

    def forward(self, inputs, target):
        bce_loss = F.binary_cross_entropy_with_logits(inputs, target, reduction='none', pos_weight=self.pos_weight)

        p_t = torch.exp(-bce_loss)
        focal_loss = (self.alpha * (1 - p_t) ** self.gamma * bce_loss)

        if self.reduction == 'mean':
            return torch.mean(focal_loss)
        elif self.reduction == 'sum':
            return torch.sum(focal_loss)
        else:
            return focal_loss

In [165]:
pos_weight = (len(data_iterator.train.dataset) - sample_per_cls) / sample_per_cls
pos_weight = torch.Tensor(pos_weight)
pos_weight[pos_weight > 4] = 4
print(pos_weight)
# Distribution-Balanced Loss
loss_fn = FocalLossWithPosWeight(alpha=1, gamma=2, pos_weight=pos_weight.to(training_args.device), reduction='mean')

tensor([2.5377, 4.0000, 4.0000, 4.0000, 4.0000, 4.0000, 4.0000, 4.0000, 1.5974,
        4.0000, 4.0000, 4.0000, 4.0000, 4.0000, 4.0000, 4.0000, 4.0000, 4.0000,
        4.0000, 4.0000, 4.0000, 4.0000, 4.0000, 4.0000, 4.0000],
       dtype=torch.float64)


In [166]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [167]:
trainer = Trainer(model, optimizer, data_iterator, loss_fn, training_args)
trainer.fit(training_args.n_epochs)

train #1: 100%|██████████████████████████████| 11/11 [00:30<00:00,  2.79s/it, epoch=1, iteration=10, loss=0.262]       


국립국어원 기준 | Threshold: 0.65 | # Precision : 0.0606 | Recall : 0.1399 | F1 : 0.0846
국립국어원 기준 | Threshold: 0.7 | # Precision : 0.0592 | Recall : 0.0631 | F1 : 0.0611
국립국어원 기준 | Threshold: 0.75 | # Precision : 0.0622 | Recall : 0.0274 | F1 : 0.0380
국립국어원 기준 | Threshold: 0.8 | # Precision : 0.0425 | Recall : 0.0067 | F1 : 0.0115


val #1: 100%|██████████████████████████████| 10/10 [00:08<00:00,  1.15it/s, epoch=1, iteration=9, loss=0.22]           


EP:1 | Model Saved on: ../saved_model/cd_new0.22.pt
국립국어원 기준 | Threshold: 0.65 | # Precision : 0.5000 | Recall : 0.0000 | F1 : 0.0000
국립국어원 기준 | Threshold: 0.7 | # Precision : 0.5000 | Recall : 0.0000 | F1 : 0.0000
국립국어원 기준 | Threshold: 0.75 | # Precision : 0.5000 | Recall : 0.0000 | F1 : 0.0000
국립국어원 기준 | Threshold: 0.8 | # Precision : 0.5000 | Recall : 0.0000 | F1 : 0.0000


train #2: 100%|██████████████████████████████| 11/11 [00:22<00:00,  2.06s/it, epoch=2, iteration=10, loss=0.235]       


...

EP:24 | Model Saved on: ../saved_model/cd_new0.0839.pt
국립국어원 기준 | Threshold: 0.65 | # Precision : 0.6871 | Recall : 0.6731 | F1 : 0.6800
국립국어원 기준 | Threshold: 0.7 | # Precision : 0.7184 | Recall : 0.6506 | F1 : 0.6828
국립국어원 기준 | Threshold: 0.75 | # Precision : 0.7476 | Recall : 0.6219 | F1 : 0.6790
국립국어원 기준 | Threshold: 0.8 | # Precision : 0.7742 | Recall : 0.5917 | F1 : 0.6708


train #25: 100%|██████████████████████████████| 11/11 [00:20<00:00,  1.88s/it, epoch=25, iteration=10, loss=0.031]     


국립국어원 기준 | Threshold: 0.65 | # Precision : 0.9076 | Recall : 0.9310 | F1 : 0.9192
국립국어원 기준 | Threshold: 0.7 | # Precision : 0.9394 | Recall : 0.9093 | F1 : 0.9241
국립국어원 기준 | Threshold: 0.75 | # Precision : 0.9604 | Recall : 0.8739 | F1 : 0.9151
국립국어원 기준 | Threshold: 0.8 | # Precision : 0.9757 | Recall : 0.8169 | F1 : 0.8893


val #25: 100%|██████████████████████████████| 10/10 [00:08<00:00,  1.23it/s, epoch=25, iteration=9, loss=0.0859]       


EP:25 | Model Saved on: ../saved_model/cd_new0.0859.pt
국립국어원 기준 | Threshold: 0.65 | # Precision : 0.6859 | Recall : 0.6742 | F1 : 0.6800
국립국어원 기준 | Threshold: 0.7 | # Precision : 0.7111 | Recall : 0.6509 | F1 : 0.6797
국립국어원 기준 | Threshold: 0.75 | # Precision : 0.7385 | Recall : 0.6259 | F1 : 0.6775
국립국어원 기준 | Threshold: 0.8 | # Precision : 0.7659 | Recall : 0.5906 | F1 : 0.6669


# Inference

In [168]:
model_args = ModelArgs()
data_args = DataArgs()
training_args = TrainingArgs()
model_name = 'cd_new0.0809'

tokenizer = AutoTokenizer.from_pretrained(model_args.huggingface_baseline)
tokenizer.add_special_tokens(special_tokens_dict)

model = CustomModel(model_args, len_tokenizer = len(tokenizer)).to(training_args.device)
model.load_state_dict(torch.load(os.path.join(TrainingArgs.save_path, f'{model_name}.pt')))
model.eval()

optimizer = AdamW(params=filter(lambda x: x.requires_grad, model.parameters()), lr=training_args.lr)
test_dataset = CustomDataset(data_args.test_dataset_path, tokenizer, max_tokens=model_args.max_tokens, is_test=True)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [169]:
# labels = [data['labels'] for data in tmp]
# labels = torch.stack(labels)

pred = []

for data in tqdm.tqdm(test_dataset):
  input_ids = data['input_ids'].unsqueeze(0).to(training_args.device)
  attention_mask = data['attention_mask'].unsqueeze(0).to(training_args.device)
  with torch.no_grad():
    logits = model.forward(input_ids=input_ids, attention_mask=attention_mask).cpu()
  pred += logits

pred = torch.stack(pred)

100%|██████████████████████████████████████████████████████████████████████████████| 2127/2127 [00:22<00:00, 95.47it/s]


In [170]:
threshold = 0.7
pred_names = []
for p in pred:
  idxs = np.where(torch.sigmoid(p).cpu() > threshold)[0].tolist()
  if not idxs:
    idxs = np.where(torch.sigmoid(p).cpu() > threshold - 0.1)[0].tolist()
    if not idxs:
      idxs = [torch.argmax(p)]
  pred_name = [[category_id2name[idx], None] for idx in idxs]
  
  pred_names.append(pred_name)

In [171]:
test_dataset.data['annotation'] = pd.Series(pred_names)

In [172]:
test_dataset.data['annotation']

0                             [[제품 전체#일반, None]]
1                            [[제품 전체#디자인, None]]
2                                [[본품#일반, None]]
3                                [[본품#일반, None]]
4                                [[본품#일반, None]]
                          ...                   
2122                            [[본품#편의성, None]]
2123                            [[본품#편의성, None]]
2124                             [[본품#품질, None]]
2125                        [[패키지/구성품#일반, None]]
2126    [[제품 전체#편의성, None], [패키지/구성품#편의성, None]]
Name: annotation, Length: 2127, dtype: object

In [173]:
print(test_dataset.data.loc[:, 'annotation'].value_counts())

[[본품#품질, None]]                                               579
[[제품 전체#일반, None]]                                            382
[[제품 전체#품질, None]]                                            244
[[본품#일반, None]]                                               174
[[제품 전체#디자인, None]]                                           138
                                                             ... 
[[제품 전체#일반, None], [본품#편의성, None]]                              1
[[제품 전체#디자인, None], [제품 전체#품질, None], [제품 전체#편의성, None]]        1
[[제품 전체#일반, None], [제품 전체#디자인, None], [패키지/구성품#디자인, None]]      1
[[제품 전체#편의성, None], [본품#편의성, None], [패키지/구성품#편의성, None]]        1
[[제품 전체#품질, None], [패키지/구성품#디자인, None]]                         1
Name: annotation, Length: 66, dtype: int64


In [174]:
test_dataset.data.to_json(os.path.join('../results/',f'{model_name}.json'), orient='records', force_ascii=False, lines=True)